In [19]:
import gzip
import pickle as pkl


# Function to load all objects from a .pkl.gz file
def load_all(file_path):
    objects = []
    with gzip.open(file_path, 'rb') as f:
        while True:
            try:
                # Load the next pickle object
                objects.append(pkl.load(f))
            except EOFError:
                # End of file reached
                break
    return objects

data = 'CIFAR10'
model = 'resnet50'
n_rounds = 1
nb_epochs = 60
seeds = 1
loss = 'Madry'

results_list = []
for active_strategy in [ 'uncertainty', 'random']: #'attack_uncertainty', 'attack', 'margin', 'entropy',
    for size in [5.0, 10.0, 25.0]: 
        for lr in [0.01, 0.001]:
            for sched in ['sched', 'nosched']:
                for seed in range(seeds):
                    try:
                        file_name = './results/{}_{}_{}_{}_{}_{}_{}_{}_{}_{}.pkl.gz'.format(loss, lr, sched, data, model, active_strategy, n_rounds, size, nb_epochs, seed)
                        with gzip.open(file_name, 'rb') as f:
                            result = load_all(file_name)[-1]
                        result['active_strategy'] = active_strategy
                        result['lr'] = lr
                        print(result)
                        results_list.append(result)
                    except:
                        print('issue')
                        pass


{'active_strategy': 'uncertainty', 'n_rounds': 1, 'size': 5.0, 'nb_epochs': 60, 'seed': 0, 'data': 'CIFAR10', 'model': 'resnet50', 'loss': 'Madry', 'sched': 'sched', 'init_clean_accuracy': 0.9365000128746033, 'card': 2500, 'final_clean_accuracy': 0.7365999817848206, 'final_PGD_accuracy': 0.3467999994754791, 'lr': 0.01}
{'active_strategy': 'uncertainty', 'n_rounds': 1, 'size': 5.0, 'nb_epochs': 60, 'seed': 0, 'data': 'CIFAR10', 'model': 'resnet50', 'loss': 'Madry', 'sched': 'nosched', 'init_clean_accuracy': 0.9365000128746033, 'card': 2500, 'final_clean_accuracy': 0.6597999930381775, 'final_PGD_accuracy': 0.35109999775886536, 'lr': 0.01}
{'active_strategy': 'uncertainty', 'n_rounds': 1, 'size': 5.0, 'nb_epochs': 60, 'seed': 0, 'data': 'CIFAR10', 'model': 'resnet50', 'loss': 'Madry', 'sched': 'sched', 'init_clean_accuracy': 0.9365000128746033, 'card': 2500, 'final_clean_accuracy': 0.8920999765396118, 'final_PGD_accuracy': 0.414000004529953, 'lr': 0.001}
{'active_strategy': 'uncertainty',

In [36]:
import pandas as pd
# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(results_list)

data = df.groupby(['active_strategy', 'size', 'sched', 'lr' ]).agg({
    'final_clean_accuracy': ['mean',  ], #std
    'final_PGD_accuracy': ['mean', ] #std
})

# Flattening the multi-level column index
data.columns = ['_'.join(col).strip() for col in data.columns.values]

# Resetting the index to move the index values into columns
data.reset_index(inplace=True)

data.to_csv('./results/means.csv', index=True)

In [42]:
data

,active_strategy,size,sched,lr,final_clean_accuracy_mean,final_PGD_accuracy_mean
0,random,5.0,nosched,0.001,0.8522,0.4675
1,random,5.0,nosched,0.01,0.5822,0.2554
2,random,5.0,sched,0.001,0.8815,0.4781
3,random,5.0,sched,0.01,0.6024,0.3040
4,random,10.0,nosched,0.001,0.7803,0.4485
5,random,10.0,nosched,0.01,0.5391,0.2331
6,random,10.0,sched,0.001,0.8731,0.5099
7,random,10.0,sched,0.01,0.5589,0.2552
8,random,25.0,nosched,0.001,0.6088,0.2757
9,random,25.0,nosched,0.01,0.5519,0.2009


In [43]:
# Convert the learning rate column to string to match keys in the lr_styles dictionary
data['lr'] = data['lr'].astype(str)

# Update the strategy_colors dictionary to include all unique strategies from the data
strategy_colors = {'random': 'red', 'uncertainty': 'blue'}

# Define line styles for learning rates
lr_styles = {'0.001': 'solid', '0.01': 'dot'}

# Reinitialize the figure with increased marker size and improved legend
fig = go.Figure()

# Add traces for each combination of sched and active_strategy
for sched in data['sched'].unique():
    for active_strategy in data['active_strategy'].unique():
        subset = data[(data['sched'] == sched) & (data['active_strategy'] == active_strategy)]
        for lr in subset['lr'].unique():
            subsubset = subset[subset['lr'] == lr]
            fig.add_trace(go.Scatter(
                x=subsubset['size'],
                y=subsubset['final_PGD_accuracy_mean'],
                mode='lines+markers',
                name=f'{active_strategy}, {sched}, {lr}',
                line=dict(color=strategy_colors[active_strategy], dash=lr_styles[lr]),
                marker=dict(symbol='circle' if sched == 'nosched' else 'square', size=10)
            ))

# Update layout for better legend visibility
fig.update_layout(
    margin=dict(l=0, r=0, t=50, b=0),
    title='Final Clean Accuracy by Size and Strategy',
    xaxis_title='Size',
    yaxis_title='Final Clean Accuracy',
    legend_title='Strategy Details',
    legend=dict(
        orientation="v",
        yanchor="top",
        y=1,
        xanchor="right",
        x=1.3,
        title_font=dict(size=12),
        font=dict(size=10)
    )
)

In [25]:
import plotly.graph_objects as go
import pandas as pd

# Load and prepare the data
new_data = pd.read_csv('results/means.csv', skiprows=2)
new_data.columns = ['active_strategy', 'size', 'final_clean_accuracy',
                    'final_clean_accuracy_std', 'final_PGD_accuracy', 'final_PGD_accuracy_std']
new_data['size'] = new_data['size'].astype(int)

for performance in ['PGD_accuracy', 'clean_accuracy' ]:
# performance = 'final_PGD_accuracy' 
# # performance = 'final_clean_accuracy'
    fig = go.Figure()

    # fig.add_trace(go.Scatter(x=[0, 50000], y=[ init_means_stds['init_'+performance]['mean'][0], init_means_stds['init_'+performance]['mean'][0] ],
    #         mode='lines', line=dict(color='rgba({},{},{},1)'.format(0,0,204), width=2, dash='dot'),
    #                     name='Init Accuracy', showlegend=True))


    performance = 'final_'+performance
    performance_std = performance + '_std'  # Corresponding standard deviation column

    strategies = {
        # 'attack_uncertainty': {'color': [204, 0, 204], 'label': 'attack-uncertainty'},  # Red
        # 'full': {'color': [0, 0, 0], 'label': 'full'},  # Red
        'uncertainty': {'color': [255, 161, 90], 'label':'uncertainty'},                    # Yellow
        'random': {'color': [25, 211, 243], 'label':'random'},                    # Cyan
        # 'entropy': {'color': [255, 102, 146], 'label':'entropy'},                  # Magenta
        # 'attack': {'color': [182, 232, 128], 'label':'attack'},                   # Orange
        # 'margin': {'color': [254, 203, 82], 'label':'margin'},
    }


    z_value = 2.576
    n = 5

    # Add a line for each strategy
    for strategy, v in strategies.items():
        r,g,b = v['color']
        c = 'rgba({},{},{},1)'.format(r,g,b)

        # Add a horizontal line for the 'full' strategy
        if strategy == 'full':
            pass
        #     full_data = new_data[new_data['active_strategy'] == 'full']
        #     # Calculate the mean PGD accuracy for the 'full' strategy
        #     full_mean_accuracy = full_data[performance].mean()
        #     fig.add_trace(go.Scatter(x=[min(new_data['size']), max(new_data['size'])], y=[full_mean_accuracy, full_mean_accuracy],
        #                             mode='lines', line=dict(color=c, width=2, dash='dash'),
        #                             name='Full Dataset Average', showlegend=True))
            
        #     strategy_data = new_data[new_data['active_strategy'] == strategy]
        #     fig.add_trace(go.Scatter( x=strategy_data['size'],  y=strategy_data[performance],  mode='lines+markers',
        #                     name=strategy, line=dict(color=c  ) ,
        #                     error_y=dict(
        #                     type='data',  symmetric=False,  
        #                     array=z_value * strategy_data[performance_std] / n,  
        #                     arrayminus=z_value * strategy_data[performance_std] / n   ), showlegend=False
        #                         ))

            
        else:
            strategy_data = new_data[new_data['active_strategy'] == strategy]
            fig.add_trace(go.Scatter(x=strategy_data['size'], y=strategy_data[performance], line=dict(color=c),
                                    error_y=dict(
                                        type='data',  symmetric=False,  
                                        array=z_value * strategy_data[performance_std] / n,  
                                        arrayminus=z_value * strategy_data[performance_std] / n   ),
                                    mode='lines+markers', name=strategy))


    title_str = 'Robust accuracy' if performance=='final_PGD_accuracy' else 'Clean Accuracy'

    # Update layout and display the plot
    fig.update_layout(
        width=600,  # Adjusted for two subplots
        height=400,
        plot_bgcolor='white',
        paper_bgcolor='white',
        margin=dict(l=0, r=0, t=50, b=0),
        # title="Box Plot of Datasets by Cases and Columns",
        title=title_str,
        xaxis_title='Size (% of training observations)',
        yaxis_title=title_str,
        legend_title='Data Selection Strategy',
        # yaxis_range=[min(new_data[performance]) - 2, max(new_data[performance]) + 2],
        showlegend=True,
        xaxis=dict(
            # type='log',
            showgrid=True,        # Enable the grid
            gridcolor='lightgray' # Set grid line color
        ),
        yaxis=dict(
            showgrid=True,        # Enable the grid
            gridcolor='lightgray', # Set grid line color
            #range=[75,94] if 'clean' in performance else [45,65]
        )
        )

    # Display the figure
    fig.show()
    # fig.write_image("./figures/results_{}_{}_{}_{}_{}.pdf".format(performance, data, model, n_rounds, nb_epochs) )
